# Day 2: Password Philosophy

## Problem

Your input is a list of passwords and the corporate policy when that password was set.

* 1-3 a: abcde
* 1-3 b: cdefg
* 2-9 c: ccccccccc

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

How many passwords are valid according to their policies?

## Part 1 Solution

Parsing will be the hardest part of this exercise. I know F# has an excellent parsing library called FParsec but I've never actually tried it before. Let's go!

In [1]:
#!fsharp
type CheckPassword = {
    First: int
    Second: int
    Letter: char
    Password: string
}

#r "nuget: FParsec"

module CheckPassword =

    let create first second letter password = {
        First = first
        Second = second
        Letter = letter
        Password = password
    }

    open FParsec

    let parse x =
        let ws = spaces
        let str x = pstring x .>> ws
        let number = pint32 .>> ws
        let matchLetter = letter .>> ws

        let parser = 
            many (
                ws >>.                      // skip whitespace
                number .>>                  // collect a number
                str "-" .>>.                // we expect the str "-"
                number .>>.                 // collect a number
                matchLetter .>>             // collect a single letter
                str ":" .>>.                // we expect the str ":"
                manySatisfy isLetter .>>    // collect a sequence of letters
                ws                          // skip whitespace
            )


        // After running the parser, we turn collected value into a CheckPassword.
        match run parser x with
        | Success (results, _, _) ->
            seq {
                for (((min, max), theLetter), password) in results do
                    create min max theLetter password
            }
        | Failure (errorMsg, _, _) -> failwith errorMsg

// Couple of examples.
// The parser handles single line inputs and multi line inputs.
// It also deals with whitespace.
[
    "1-3 a: abcde"
    "1-3 b: cdefg"
    "2-9 c: ccccccccc"
    "1-3a:abcde"
    "1-3b:cdefg"
    "2-9c:ccccccccc"
    """
    1-3 a: abcde
    1-3 b: cdefg
    2-9 c: ccccccccc


    2-9c:ccccccccc


    """
] |> List.map (fun input -> {| Input = input; Parsed = CheckPassword.parse input|})

index,Input,Parsed
0,1-3 a: abcde,"[ { First = 1 Second = 3 Letter = 'a' Password = ""abcde"" } ]"
1,1-3 b: cdefg,"[ { First = 1 Second = 3 Letter = 'b' Password = ""cdefg"" } ]"
2,2-9 c: ccccccccc,"[ { First = 2 Second = 9 Letter = 'c' Password = ""ccccccccc"" } ]"
3,1-3a:abcde,"[ { First = 1 Second = 3 Letter = 'a' Password = ""abcde"" } ]"
4,1-3b:cdefg,"[ { First = 1 Second = 3 Letter = 'b' Password = ""cdefg"" } ]"
5,2-9c:ccccccccc,"[ { First = 2 Second = 9 Letter = 'c' Password = ""ccccccccc"" } ]"
6,1-3 a: abcde 1-3 b: cdefg 2-9 c: ccccccccc 2-9c:ccccccccc,"[ { First = 1 Second = 3 Letter = 'a' Password = ""abcde"" }, { First = 1 Second = 3 Letter = 'b' Password = ""cdefg"" }, { First = 2 Second = 9 Letter = 'c' Password = ""ccccccccc"" }, { First = 2 Second = 9 Letter = 'c' Password = ""ccccccccc"" } ]"


Now that we're able to parse the input, let's move on to solving the puzzle.

In [1]:
#!fsharp
module Seq =
    let countChar (needle: char) (haystack: string) = 
        seq {
            for c in haystack do
                if c = needle then c else ()
        } |> Seq.length

let check ({ First = first; Second = second; Letter = letter; Password = password }: CheckPassword): bool =
    let cnt = Seq.countChar letter password
    cnt >= first && cnt <= second

"""
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""
|> CheckPassword.parse
|> Seq.filter check
|> Seq.length


2

## Part 2

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

In [1]:
#!fsharp
module Seq =
    // item1 helps with 1 based indexes.
    let item1 index = Seq.item (index - 1)

let check ({ First = first; Second = second; Letter = letter; Password = password }: CheckPassword): bool =
    let letterAtFirstPos = Seq.item1 first password
    let letterAtSecondPos = Seq.item1 second password
    (letterAtFirstPos = letter) <> (letterAtSecondPos = letter)

"""
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""
|> CheckPassword.parse
|> Seq.filter check
|> Seq.length

1